## Setup

In [107]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
# set random seed to ensure that results are repeatable
np.random.seed(1)

# load data

In [108]:
UniversalBank = pd.read_csv("./UniversalBank.csv")

UniversalBank.head(3)

,ID,Age,Experience,Income,ZIP_Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD_Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0


In [109]:
UniversalBank.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP_Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD_Account            0
Online                0
CreditCard            0
dtype: int64

In [110]:
UniversalBank=UniversalBank.drop(["ID"], axis=1)

In [111]:
UniversalBank.head()

,Age,Experience,Income,ZIP_Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD_Account,Online,CreditCard
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [112]:
UniversalBank=pd.get_dummies(UniversalBank, prefix=['Education','Family'], columns=['Education','Family'])
UniversalBank.head()

,Age,Experience,Income,ZIP_Code,CCAvg,Mortgage,Personal Loan,Securities Account,CD_Account,Online,CreditCard,Education_1,Education_2,Education_3,Family_1,Family_2,Family_3,Family_4
0,25,1,49,91107,1.6,0,0,1,0,0,0,1,0,0,0,0,0,1
1,45,19,34,90089,1.5,0,0,1,0,0,0,1,0,0,0,0,1,0
2,39,15,11,94720,1.0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,35,9,100,94112,2.7,0,0,0,0,0,0,0,1,0,1,0,0,0
4,35,8,45,91330,1.0,0,0,0,0,0,1,0,1,0,0,0,0,1


In [113]:
UniversalBank.columns

Index(['Age', 'Experience', 'Income', 'ZIP_Code', 'CCAvg', 'Mortgage',
       'Personal Loan', 'Securities Account', 'CD_Account', 'Online',
       'CreditCard', 'Education_1', 'Education_2', 'Education_3', 'Family_1',
       'Family_2', 'Family_3', 'Family_4'],
      dtype='object')

## Split data (train/test)

In [114]:
X = UniversalBank[['Age', 'Experience', 'Income', 'ZIP_Code', 'CCAvg', 'Mortgage',
       'Personal Loan', 'Securities Account', 'Online',
       'CreditCard', 'Education_1', 'Education_2', 'Education_3', 'Family_1',
       'Family_2', 'Family_3', 'Family_4']]
y = UniversalBank[['CD_Account']]

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
scaler = StandardScaler()

In [58]:
list_21 =['Age', 'Experience', 'Income', 'ZIP_Code', 'CCAvg', 'Mortgage']

In [59]:
X_train[list_21] = scaler.fit_transform(X_train[list_21])
X_test[list_21] = scaler.transform(X_test[list_21])

## Model the data

In [90]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## Logistic Regression model

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [62]:
log_reg_model = LogisticRegression(max_iter=900)
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [91]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636


# Change to liblinear solver

In [64]:
log_reg_liblin_model = LogisticRegression(solver='liblinear').fit(X_train, np.ravel(y_train))

In [92]:
model_preds = log_reg_liblin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"liblinear logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636
0,liblinear logistic,0.973714,1.0,0.585586,0.738636


## L1 Regularization

In [66]:
log_reg_L1_model = LogisticRegression(solver='liblinear', penalty='l1')
_ = log_reg_L1_model.fit(X_train, np.ravel(y_train))

In [93]:
model_preds = log_reg_L1_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"L1 logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636
0,liblinear logistic,0.973714,1.0,0.585586,0.738636
0,L1 logistic,0.973714,1.0,0.585586,0.738636


## L2 Regularization

In [68]:
log_reg_L2_model = LogisticRegression(penalty='l2', max_iter=1000)
_ = log_reg_L2_model.fit(X_train, np.ravel(y_train))

In [94]:
model_preds = log_reg_L2_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"L2 logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636
0,liblinear logistic,0.973714,1.0,0.585586,0.738636
0,L1 logistic,0.973714,1.0,0.585586,0.738636
0,L2 logistic,0.973714,1.0,0.585586,0.738636


## Elastic Net Regularization

In [70]:
log_reg_elastic_model = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.5, max_iter=1000)
_ = log_reg_elastic_model.fit(X_train, np.ravel(y_train))

C:\Users\akhil\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [95]:
model_preds = log_reg_elastic_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Elestic logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636
0,liblinear logistic,0.973714,1.0,0.585586,0.738636
0,L1 logistic,0.973714,1.0,0.585586,0.738636
0,L2 logistic,0.973714,1.0,0.585586,0.738636
0,Elestic logistic,0.973714,1.0,0.585586,0.738636


In [103]:
score_measure = "recall"
LR=LogisticRegression()
kfolds = 5
param_grid = {'C': [0.1, 1, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','lasso','elastic']} 
  
grid = RandomizedSearchCV(LR, param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END ......C=10, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ......C=10, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ......C=10, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ......C=10, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ......C=10, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, penalty=l2, solver=liblinear;, score=0.942 total time=   0.0s
[CV 2/5] END C=0.001, penalty=l2, solver=libl

C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 4/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.966 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.951 total time=   0.0s
[CV 1/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .......C=1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.001, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.001, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.001, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.001, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.001, penalty=elastic, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END ..C

C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 3/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.968 total time=   0.0s
[CV 4/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.972 total time=   0.0s
[CV 5/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.963 total time=   0.0s
The best recall score is 0.9673846153846155
... with parameters: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\akhil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\akhil\anaconda3\lib\site-pac

In [104]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [105]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'solver' : [ 'lbfgs', 'liblinear'],
              'penalty' : ['l1','l2','lasso','elastic']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END .....C=0.1, penalty=l1, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.982 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.982 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l1, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.971 total time=   0.0s


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

[CV 2/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.963 total time=   0.0s
[CV 3/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.968 total time=   0.0s
[CV 4/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.972 total time=   0.0s
[CV 5/5] END ...C=0.1, penalty=l2, solver=lbfgs;, score=0.963 total time=   0.0s


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.957 total time=   0.0s
[CV 2/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.955 total time=   0.0s
[CV 3/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.954 total time=   0.0s
[CV 4/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.966 total time=   0.0s
[CV 5/5] END C=0.1, penalty=l2, solver=liblinear;, score=0.951 total time=   0.0s
[CV 1/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ..C=0.1, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.1,

...

C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 4/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.980 total time=   0.1s
[CV 5/5] END ....C=10, penalty=l2, solver=lbfgs;, score=0.983 total time=   0.0s
[CV 1/5] END C=10, penalty=l2, solver=liblinear;, score=0.982 total time=   0.0s
[CV 2/5] END C=10, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 3/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 4/5] END C=10, penalty=l2, solver=liblinear;, score=0.980 total time=   0.0s
[CV 5/5] END C=10, penalty=l2, solver=liblinear;, score=0.983 total time=   0.0s
[CV 1/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END ...C=10, penalty=lasso, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=10, penalty=lasso, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=10, penal

C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\akhil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 120.
The score on these train-test partitions for these parameters

In [106]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

## SVM classification model using linear kernal

In [72]:
from sklearn.svm import SVC

In [73]:
svm_lin_model = SVC(kernel="linear")
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [96]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.0,0.585586,0.738636
0,liblinear logistic,0.973714,1.0,0.585586,0.738636
0,L1 logistic,0.973714,1.0,0.585586,0.738636
0,L2 logistic,0.973714,1.0,0.585586,0.738636
0,Elestic logistic,0.973714,1.0,0.585586,0.738636
0,svm with linear kernel,0.973714,1.0,0.585586,0.738636


## SVM classification model using rbf kernal

In [75]:
svm_rbf_model = SVC(kernel="rbf", C=10)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [97]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.000000,0.585586,0.738636
0,liblinear logistic,0.973714,1.000000,0.585586,0.738636
0,L1 logistic,0.973714,1.000000,0.585586,0.738636
0,L2 logistic,0.973714,1.000000,0.585586,0.738636
0,Elestic logistic,0.973714,1.000000,0.585586,0.738636
0,svm with linear kernel,0.973714,1.000000,0.585586,0.738636
0,svm with rbf kernel,0.967429,0.864865,0.576577,0.691892


## SVM classification model using polynomial kernal

In [77]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [98]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.000000,0.585586,0.738636
0,liblinear logistic,0.973714,1.000000,0.585586,0.738636
0,L1 logistic,0.973714,1.000000,0.585586,0.738636
0,L2 logistic,0.973714,1.000000,0.585586,0.738636
0,Elestic logistic,0.973714,1.000000,0.585586,0.738636
0,svm with linear kernel,0.973714,1.000000,0.585586,0.738636
0,svm with rbf kernel,0.967429,0.864865,0.576577,0.691892
0,svm with polynomial kernel,0.956000,0.660377,0.630631,0.645161


In [ ]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10], 
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['linear','poly','rbf']} 
  
grid = RandomizedSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search - SVM ", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [ ]:
score_measure = "recall"
kfolds = 5
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

In [ ]:
c_matrix = confusion_matrix(y_test, grid.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search SVM ", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])


## DTree Classifer

In [79]:
#Conduct an initial random search across a wide range of possible parameters.

In [80]:
from sklearn.tree import DecisionTreeClassifier

In [81]:
dt=DecisionTreeClassifier()
dt=dt.fit(X_train, y_train)

In [99]:
c_matrix = confusion_matrix(y_test, dt.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.000000,0.585586,0.738636
0,liblinear logistic,0.973714,1.000000,0.585586,0.738636
0,L1 logistic,0.973714,1.000000,0.585586,0.738636
0,L2 logistic,0.973714,1.000000,0.585586,0.738636
0,Elestic logistic,0.973714,1.000000,0.585586,0.738636
0,svm with linear kernel,0.973714,1.000000,0.585586,0.738636
0,svm with rbf kernel,0.967429,0.864865,0.576577,0.691892
0,svm with polynomial kernel,0.956000,0.660377,0.630631,0.645161
0,DT,0.949143,0.591667,0.639640,0.614719


# Randomized Search

In [83]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

In [84]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,60), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.6960863697705804
... with parameters: {'min_samples_split': 28, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 189, 'max_depth': 31, 'criterion': 'entropy'}


C:\Users\akhil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
55 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\akhil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\akhil\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\akhil\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [85]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.9708571 Precision=0.9054054 Recall=0.6036036 F1=0.7243243


In [100]:
c_matrix = confusion_matrix(y_test,rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT_random", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.000000,0.585586,0.738636
0,liblinear logistic,0.973714,1.000000,0.585586,0.738636
0,L1 logistic,0.973714,1.000000,0.585586,0.738636
0,L2 logistic,0.973714,1.000000,0.585586,0.738636
0,Elestic logistic,0.973714,1.000000,0.585586,0.738636
0,svm with linear kernel,0.973714,1.000000,0.585586,0.738636
0,svm with rbf kernel,0.967429,0.864865,0.576577,0.691892
0,svm with polynomial kernel,0.956000,0.660377,0.630631,0.645161
0,DT,0.949143,0.591667,0.639640,0.614719
0,DT_random,0.970857,0.905405,0.603604,0.724324


## Grid Search

In [87]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(25,30),  
    'min_samples_leaf': np.arange(1,5),
    'min_impurity_decrease': np.arange(0.0000, 0.0004, 0.0001),
    'max_leaf_nodes': np.arange(187,192), 
    'max_depth': np.arange(29,33), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
The best recall score is 0.7116059379217273
... with parameters: {'criterion': 'entropy', 'max_depth': 29, 'max_leaf_nodes': 187, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 25}


In [88]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.9668571 Precision=0.8441558 Recall=0.5855856 F1=0.6914894


In [101]:
c_matrix = confusion_matrix(y_test,grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"DT_grid_Search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.973714,1.000000,0.585586,0.738636
0,liblinear logistic,0.973714,1.000000,0.585586,0.738636
0,L1 logistic,0.973714,1.000000,0.585586,0.738636
0,L2 logistic,0.973714,1.000000,0.585586,0.738636
0,Elestic logistic,0.973714,1.000000,0.585586,0.738636
0,svm with linear kernel,0.973714,1.000000,0.585586,0.738636
0,svm with rbf kernel,0.967429,0.864865,0.576577,0.691892
0,svm with polynomial kernel,0.956000,0.660377,0.630631,0.645161
0,DT,0.949143,0.591667,0.639640,0.614719
0,DT_random,0.970857,0.905405,0.603604,0.724324


## Discussion section

In [ ]:
# we dropped the ID column form the Table 

In [ ]:
# we did One-Hot-Encoding for the columns Education and Family

In [ ]:
#As we can attributes, we can say that Decision tress has the best recall among the other models with 63.9

In [ ]:
s